In [8]:
# TODO - find a way to make this install the poetry enviorment
import os
os.chdir("C:\\Users\yuvmi\PycharmProjects\SequenceModelLab")
!poetry install

Installing dependencies from lock file

No dependencies to install or update


In [9]:
import numpy as np
import re

In [10]:
import os
import glob
import pickle
from collections import defaultdict
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FFMpegWriter, PillowWriter
import torch

In [11]:
from src.visualizations.vis_utils import animate_SSM_state_dynamics

# Analysing the best results dynamics

## Reading result logs and saving best results

In [12]:
INSIGNIFICANT_LOSS_CHANGE = 0.02

class Result():
    
    def __init__(self, data):
        losses = np.array([l.entity for l in data["loss"]])
        l_epochs = np.array([l.epoch_logged for l in data["loss"]])
        if len(losses) == 0:
            self.min_loss = None
            return
        exploded = (np.argmax(losses) == len(losses) - 1)
        min_loss = np.min(losses)
        best_ker = data["kernel"][np.argmin(losses)-1].entity
    
        
        if np.isnan(min_loss):
            self.min_loss = None
            return
        elif np.isinf(min_loss):
            self.min_loss = None
            return
        else:
            self.min_loss = min_loss
        self.exploded = exploded
        self.best_ker = best_ker
        self.epoch_best_res = np.argmin(losses)
        self.best_A = data["A"][np.argmin(losses)-1].entity
        self.best_B = data["B"][np.argmin(losses)-1].entity
        self.best_C = data["C"][np.argmin(losses)-1].entity
        self.train_len = len(losses)
        self.effective_train_len = np.where(losses<INSIGNIFICANT_LOSS_CHANGE+self.min_loss)[0][0]
        
    
    def __bool__(self):
        return self.min_loss is not None

In [14]:
def string_to_dict(string, pattern):
    regex = re.sub(r'{(.+?)}', r'(?P<_\1>.+)', pattern)
    values = list(re.search(regex, string).groups())
    keys = re.findall(r'{(.+?)}', pattern)
    _dict = dict(zip(keys, values))
    return _dict

In [36]:
def get_hippo_params(filepath):
    file_name = os.path.basename(file_path)
    params = string_to_dict(filepath,"hippoh_h{hidden}_l{lag}_lr{lr}_dt{dt}_o{opt}")
    params["exp_name"] = "hippo"
    return params

def get_ssm_params(filepath):
    file_name = os.path.basename(file_path)
    params = string_to_dict(filepath,"ssm_h{hidden}_l{lag}_lr{lr}_n{noise}_d{diag_init}_b{BC_std}_o{opt}")
    params["exp_name"] = "ssm"
    return params

def get_hippo_no_params(filepath):
    file_name = os.path.basename(file_path)
    params = string_to_dict(filepath,"hippo_no{no}_h{hidden}_l{lag}_lr{lr}_dt{dt}_o{opt}")
    params["exp_name"] = "hippo_no"
    return params

def get_rot_params(filepath):
    file_name = os.path.basename(file_path)
    params = string_to_dict(filepath,"rot_{rot_type}_h{hidden}_l{lag}_lr{lr}_o{opt}")
    params["exp_name"] = "rot"
    return params
    

In [37]:
experiments_results = defaultdict(list)
files = []

In [38]:
#experiment_file_regexs = [r"../results/grid/*"]
experiment_file_regexs = [r".\results\\grid\\*"]

In [39]:
for experiment_file_regex in experiment_file_regexs:
    for file_path in tqdm(glob.glob(experiment_file_regex)):
        if "filepart" in file_path:
            print(file_path)
            continue
        
        file_name = os.path.basename(file_path)
        if file_name in files:
            continue
        
        
        if "hippo_no" in file_name:
            params = get_hippo_no_params(file_path)
        elif "hippo" in file_name:
            params = get_hippo_params(file_path)
        elif "ssm" in file_name:
            params = get_ssm_params(file_path)
        elif "rot" in file_name:
            params = get_rot_params(file_path)
        else:
            raise
            
        with open(file_path,"rb") as f:
            try:
                data = pickle.load(f)
            except:
                print(file_path)
                continue
            result = Result(data)
            if result:
                experiments_results[params["exp_name"]].append({"result":Result(data),"params":params})
            else:
                print(file_path)
                
        files.append(file_name)

  0%|          | 0/1001 [00:00<?, ?it/s]

.\results\\grid\ssm_h64_l60_lr0.0001_n0.1_d0.93_b0.001_oSGD
.\results\\grid\ssm_h64_l60_lr0.0001_n0.1_d0.93_b0.01_oSGD
.\results\\grid\ssm_h64_l60_lr0.0001_n0.1_d0.93_b0.1_oSGD
.\results\\grid\ssm_h64_l60_lr0.0001_n0.1_d0.93_b1_oSGD
.\results\\grid\ssm_h64_l60_lr0.0001_n0.1_d0.95_b0.001_oSGD
.\results\\grid\ssm_h64_l60_lr0.0001_n0.1_d0.95_b0.01_oSGD
.\results\\grid\ssm_h64_l60_lr0.0001_n0.1_d0.95_b0.1_oSGD
.\results\\grid\ssm_h64_l60_lr0.0001_n0.1_d0.95_b1_oSGD
.\results\\grid\ssm_h64_l60_lr0.0001_n0.1_d0.97_b0.001_oSGD
.\results\\grid\ssm_h64_l60_lr0.0001_n0.1_d0.97_b0.01_oSGD
.\results\\grid\ssm_h64_l60_lr0.0001_n0.1_d0.97_b0.1_oSGD
.\results\\grid\ssm_h64_l60_lr0.0001_n0.1_d0.97_b1_oSGD
.\results\\grid\ssm_h64_l60_lr0.0001_n0.1_d0.9_b0.001_oSGD
.\results\\grid\ssm_h64_l60_lr0.0001_n0.1_d0.9_b0.01_oSGD
.\results\\grid\ssm_h64_l60_lr0.0001_n0.1_d0.9_b0.1_oSGD
.\results\\grid\ssm_h64_l60_lr0.0001_n0.1_d0.9_b1_oSGD
.\results\\grid\ssm_h64_l60_lr0.0005_n0.1_d0.93_b0.001_oSGD
.\results\\g

.\results\\grid\ssm_h64_l60_lr1e-05_n0.1_d0.93_b0.001_oSGD
.\results\\grid\ssm_h64_l60_lr1e-05_n0.1_d0.93_b0.01_oSGD
.\results\\grid\ssm_h64_l60_lr1e-05_n0.1_d0.93_b0.1_oSGD
.\results\\grid\ssm_h64_l60_lr1e-05_n0.1_d0.93_b1_oSGD
.\results\\grid\ssm_h64_l60_lr1e-05_n0.1_d0.95_b0.001_oSGD
.\results\\grid\ssm_h64_l60_lr1e-05_n0.1_d0.95_b0.01_oSGD
.\results\\grid\ssm_h64_l60_lr1e-05_n0.1_d0.95_b0.1_oSGD
.\results\\grid\ssm_h64_l60_lr1e-05_n0.1_d0.95_b1_oSGD
.\results\\grid\ssm_h64_l60_lr1e-05_n0.1_d0.97_b0.001_oSGD
.\results\\grid\ssm_h64_l60_lr1e-05_n0.1_d0.97_b0.01_oSGD
.\results\\grid\ssm_h64_l60_lr1e-05_n0.1_d0.97_b0.1_oSGD
.\results\\grid\ssm_h64_l60_lr1e-05_n0.1_d0.97_b1_oSGD
.\results\\grid\ssm_h64_l60_lr1e-05_n0.1_d0.9_b0.001_oSGD
.\results\\grid\ssm_h64_l60_lr1e-05_n0.1_d0.9_b0.01_oSGD
.\results\\grid\ssm_h64_l60_lr1e-05_n0.1_d0.9_b0.1_oSGD
.\results\\grid\ssm_h64_l60_lr1e-05_n0.1_d0.9_b1_oSGD
.\results\\grid\ssm_h64_l60_lr5e-05_n0.0001_d0.93_b0.001_oSGD.filepart


In [40]:
[k for k in experiments_results.keys()]

['hippo', 'hippo_no', 'rot', 'ssm']

## Analysis

In [47]:
res = {}
for exp in experiments_results.keys():
    res[exp] = {
        "min_losses":[v["result"].min_loss for v in experiments_results[exp]],
        "params":[v["params"] for v in experiments_results[exp]],
        "train_len":[v["result"].effective_train_len for v in experiments_results[exp]],
        "best_res":[v["result"].epoch_best_res for v in experiments_results[exp]]
    }

In [48]:
def print_by_order(res,n):
    arg_min_loss = np.argsort(res["min_losses"])
    sorted_params = [res["params"][i] for i in arg_min_loss]
    sorted_min_loss = [res["min_losses"][i] for i in arg_min_loss]
    sorted_train_len = [res["train_len"][i] for i in arg_min_loss]
    sorted_best_res_idx = [res["best_res"][i] for i in arg_min_loss]
    
    for i in range(n):
        print(sorted_min_loss[i],sorted_best_res_idx[i], sorted_train_len[i],sorted_params[i])

In [49]:
r = res["hippo"]
print_by_order(r,10)

0.11778785288333893 3999 3400 {'hidden': '64', 'lag': '60', 'lr': '0.001', 'dt': '0.1', 'opt': 'adam', 'exp_name': 'hippo'}
0.15002372860908508 3999 2631 {'hidden': '64', 'lag': '60', 'lr': '0.001', 'dt': '0.05', 'opt': 'adam', 'exp_name': 'hippo'}
0.16897200047969818 3919 2622 {'hidden': '64', 'lag': '60', 'lr': '0.01', 'dt': '0.05', 'opt': 'adam', 'exp_name': 'hippo'}
0.1800093799829483 3999 2796 {'hidden': '64', 'lag': '60', 'lr': '0.0001', 'dt': '0.05', 'opt': 'adam', 'exp_name': 'hippo'}
0.21781492233276367 3999 2422 {'hidden': '64', 'lag': '60', 'lr': '0.0001', 'dt': '0.1', 'opt': 'adam', 'exp_name': 'hippo'}
0.28899896144866943 3999 3569 {'hidden': '64', 'lag': '60', 'lr': '0.0001', 'dt': '0.1', 'opt': 'SGD', 'exp_name': 'hippo'}
0.30375415086746216 3999 1864 {'hidden': '64', 'lag': '60', 'lr': '0.0001', 'dt': '0.05', 'opt': 'SGD', 'exp_name': 'hippo'}
0.33235353231430054 3999 3428 {'hidden': '64', 'lag': '60', 'lr': '5e-05', 'dt': '0.05', 'opt': 'SGD', 'exp_name': 'hippo'}
0.46

In [50]:
r = res["hippo_no"]
print_by_order(r,10)

0.5774000883102417 3999 3304 {'no': 'a', 'hidden': '64', 'lag': '60', 'lr': '0.1', 'dt': '0.1', 'opt': 'adam', 'exp_name': 'hippo_no'}
0.6044909954071045 3999 2668 {'no': 'a', 'hidden': '64', 'lag': '60', 'lr': '0.001', 'dt': '0.1', 'opt': 'adam', 'exp_name': 'hippo_no'}
0.6343033313751221 3994 1667 {'no': 'ac', 'hidden': '64', 'lag': '60', 'lr': '0.001', 'dt': '0.1', 'opt': 'adam', 'exp_name': 'hippo_no'}
0.6353905200958252 3999 1977 {'no': 'ac', 'hidden': '64', 'lag': '60', 'lr': '0.01', 'dt': '0.1', 'opt': 'adam', 'exp_name': 'hippo_no'}
0.648389995098114 3999 3009 {'no': 'a', 'hidden': '64', 'lag': '60', 'lr': '0.01', 'dt': '0.1', 'opt': 'adam', 'exp_name': 'hippo_no'}
0.6538387537002563 3995 2354 {'no': 'ac', 'hidden': '64', 'lag': '60', 'lr': '0.1', 'dt': '0.1', 'opt': 'adam', 'exp_name': 'hippo_no'}
0.6774471998214722 3999 2895 {'no': 'ac', 'hidden': '64', 'lag': '60', 'lr': '0.0001', 'dt': '0.1', 'opt': 'adam', 'exp_name': 'hippo_no'}
0.6850076913833618 3999 2722 {'no': 'a', 'h

In [45]:
r = res["ssm"]
print_by_order(r,10)

0.6116424798965454 3999 4000 {'hidden': '64', 'lag': '60', 'lr': '0.001', 'noise': '0.001', 'diag_init': '0.93', 'BC_std': '0.1', 'opt': 'adam', 'exp_name': 'ssm'}
0.6155003309249878 3999 4000 {'hidden': '64', 'lag': '60', 'lr': '0.001', 'noise': '0.01', 'diag_init': '0.9', 'BC_std': '0.1', 'opt': 'adam', 'exp_name': 'ssm'}
0.6219192743301392 3999 4000 {'hidden': '64', 'lag': '60', 'lr': '0.001', 'noise': '0.001', 'diag_init': '0.9', 'BC_std': '0.1', 'opt': 'adam', 'exp_name': 'ssm'}
0.6245056390762329 3999 4000 {'hidden': '64', 'lag': '60', 'lr': '0.001', 'noise': '0.01', 'diag_init': '0.9', 'BC_std': '0.01', 'opt': 'adam', 'exp_name': 'ssm'}
0.6301802396774292 3999 4000 {'hidden': '64', 'lag': '60', 'lr': '0.001', 'noise': '0.01', 'diag_init': '0.9', 'BC_std': '0.001', 'opt': 'adam', 'exp_name': 'ssm'}
0.6369850635528564 3990 4000 {'hidden': '64', 'lag': '60', 'lr': '0.001', 'noise': '0.0001', 'diag_init': '0.93', 'BC_std': '0.1', 'opt': 'adam', 'exp_name': 'ssm'}
0.6436591148376465 

In [46]:
r = res["rot"]
print_by_order(r,10)

1.3258532726573291e-12 1344 2641 {'rot_type': 'eq', 'hidden': '64', 'lag': '60', 'lr': '0.001', 'opt': 'adam', 'exp_name': 'rot'}
0.030712872743606567 2782 2783 {'rot_type': 'eq', 'hidden': '64', 'lag': '60', 'lr': '0.0001', 'opt': 'adam', 'exp_name': 'rot'}
0.033999279141426086 3999 4000 {'rot_type': 'eq', 'hidden': '64', 'lag': '60', 'lr': '0.005', 'opt': 'SGD', 'exp_name': 'rot'}
0.06051699444651604 3999 4000 {'rot_type': 'eq', 'hidden': '64', 'lag': '60', 'lr': '0.01', 'opt': 'SGD', 'exp_name': 'rot'}
0.09439077973365784 2721 2722 {'rot_type': 'eq', 'hidden': '64', 'lag': '60', 'lr': '0.001', 'opt': 'SGD', 'exp_name': 'rot'}
0.12642396986484528 3999 4000 {'rot_type': 'eq', 'hidden': '64', 'lag': '60', 'lr': '0.0005', 'opt': 'SGD', 'exp_name': 'rot'}
0.197043776512146 3999 4000 {'rot_type': 'eq', 'hidden': '64', 'lag': '60', 'lr': '0.0001', 'opt': 'SGD', 'exp_name': 'rot'}
0.38720840215682983 3999 4000 {'rot_type': 'eq', 'hidden': '64', 'lag': '60', 'lr': '5e-05', 'opt': 'SGD', 'exp

In [47]:
# hippo_min_loss = [v["result"].min_loss for v in experiments_results["hippo"]]
# hippo_min_params = [v["params"] for v in experiments_results["hippo"]]
# hippo_train_len = [v["result"].train_len for v in experiments_results["hippo"]]
# hippo_best_res = [v["result"].epoch_best_res for v in experiments_results["hippo"]]

# ssm_min_loss = [v["result"].min_loss for v in experiments_results["ssm"]]
# ssm_min_params = [v["params"] for v in experiments_results["ssm"]]
# ssm_train_len = [v["result"].train_len for v in experiments_results["ssm"]]
# ssm_best_res = [v["result"].epoch_best_res for v in experiments_results["ssm"]]

# hippo_no_min_loss = [v["result"].min_loss for v in experiments_results["hippo_no"]]
# hippo_no_min_params = [v["params"] for v in experiments_results["hippo_no"]]
# hippo_no_train_len = [v["result"].train_len for v in experiments_results["hippo_no"]]
# hippo_no_best_res = [v["result"].epoch_best_res for v in experiments_results["hippo_no"]]

In [50]:
# arg_min_loss = np.argsort(hippo_min_loss)
# hippo_soted_params = [hippo_min_params[i] for i in arg_min_loss]
# sorted_min_loss = [hippo_min_loss[i] for i in arg_min_loss]
# sorted_train_len = [hippo_train_len[i] for i in arg_min_loss]
# sorted_best_res_idx = [hippo_best_res[i] for i in arg_min_loss]
# for i in range(30):
#     print(sorted_min_loss[i],sorted_best_res_idx[i], sorted_train_len[i],hippo_soted_params[i])

0.8189658522605896 999 1001 {'lag': '128', 'lr': '0.0001', 'dt': '0.005', 'opt': 'adam', 'exp_name': 'hippo'}
0.8189658522605896 999 1001 {'lag': '128', 'lr': '0.0001', 'dt': '0.005', 'opt': 'adam', 'exp_name': 'hippo'}
0.8189658522605896 999 1001 {'lag': '128', 'lr': '0.0001', 'dt': '0.005', 'opt': 'adam', 'exp_name': 'hippo'}
0.8215049505233765 703 1001 {'lag': '128', 'lr': '0.001', 'dt': '0.005', 'opt': 'adam', 'exp_name': 'hippo'}
0.8215049505233765 703 1001 {'lag': '128', 'lr': '0.001', 'dt': '0.005', 'opt': 'adam', 'exp_name': 'hippo'}
0.8215049505233765 703 1001 {'lag': '128', 'lr': '0.001', 'dt': '0.005', 'opt': 'adam', 'exp_name': 'hippo'}
0.8644284009933472 1000 1001 {'lag': '128', 'lr': '0.0001', 'dt': '0.005', 'opt': 'SGD', 'exp_name': 'hippo'}
0.8644284009933472 1000 1001 {'lag': '128', 'lr': '0.0001', 'dt': '0.005', 'opt': 'SGD', 'exp_name': 'hippo'}
0.8644284009933472 1000 1001 {'lag': '128', 'lr': '0.0001', 'dt': '0.005', 'opt': 'SGD', 'exp_name': 'hippo'}
0.86519926786

In [53]:
# arg_min_loss = np.argsort(ssm_min_loss)
# ssm_soted_params = [ssm_min_params[i] for i in arg_min_loss]
# sorted_min_loss = [ssm_min_loss[i] for i in arg_min_loss]
# sorted_train_len = [ssm_train_len[i] for i in arg_min_loss]
# sorted_best_res_idx = [ssm_best_res[i] for i in arg_min_loss]
# for i in range(10):
#     print(sorted_min_loss[i],sorted_best_res_idx[i],sorted_train_len[i],ssm_soted_params[i])

0.9113898277282715 574 1001 {'lag': '128', 'lr': '0.001', 'noise': '0.01', 'diag_init': '0.9', 'BC_std': '0.1', 'opt': 'adam', 'exp_name': 'ssm'}
0.9152887463569641 956 1001 {'lag': '128', 'lr': '0.001', 'noise': '0.0001', 'diag_init': '0.95', 'BC_std': '0.1', 'opt': 'adam', 'exp_name': 'ssm'}
0.915553867816925 1000 1001 {'lag': '128', 'lr': '0.001', 'noise': '0.001', 'diag_init': '0.95', 'BC_std': '0.1', 'opt': 'adam', 'exp_name': 'ssm'}
0.9213181734085083 1000 1001 {'lag': '128', 'lr': '0.0001', 'noise': '0.01', 'diag_init': '0.9', 'BC_std': '0.1', 'opt': 'adam', 'exp_name': 'ssm'}
0.9221504330635071 451 1001 {'lag': '128', 'lr': '0.01', 'noise': '0.01', 'diag_init': '0.9', 'BC_std': '0.1', 'opt': 'adam', 'exp_name': 'ssm'}
0.9231618046760559 1058 1061 {'lag': '128', 'lr': '0.001', 'noise': '0.0001', 'diag_init': '0.97', 'BC_std': '0.1', 'opt': 'adam', 'exp_name': 'ssm'}
0.9261329770088196 1156 1157 {'lag': '128', 'lr': '0.001', 'noise': '0.01', 'diag_init': '0.93', 'BC_std': '0.01',